# Introduction

# Configure Azure ML workspace

For this project we will be using an Azure Machine Learning Notebook VM, therefore we can skip setting up the environment.

To start we need to initialize our workspace and create a Azule ML experiment.

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# name of CPU cluster
amlcompute_cluster_name = "cpu-cluster"
vm_size = 'STANDARD_D2_V2'
max_nodes = 4

compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,max_nodes=max_nodes)
aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###

# REF: https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling?view=azure-ml-py

ps = RandomParameterSampling({
    '--C': choice(0.01, 0.1, 0.2, 0.5, 0.7, 1.0)
    '--max_iter': choice(range(10,110,10))
    }
)


# Specify a Policy
# policy = ### YOUR CODE HERE ###

# REF: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#early-termination

# options

policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5) # try this
# policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###

# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py

# Create a SKLearn estimator for use with train.py

est = SKLearn( 
    source_directory='.',
    compute_target=aml_compute,
    vm_size=vm_size, 
    vm_priority='lowpriority'
    entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###

# REF: https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py

hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=early_termination_policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4
                                    )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###